## Transfer Learning using the MobileNetV2 convolutional base

In [2]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import save_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [32]:
dir="C:\\Users\\davor\\IA\\GarbageClassification\\data\\Garbageclassification\\Garbageclassification"
classes=["cardboard","glass","metal","paper","plastic","trash"]

In [93]:
def load_data(path,size):
    images = []
    labels = []
    class_names = []
    
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, size)
                    images.append(img)
                    labels.append(classes.index(class_name))
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

## load data

In [95]:
imgs_data,labels_data = load_data(dir,(224,224))

In [97]:
print(imgs_data.shape,labels_data.shape)

(2527, 224, 224, 3) (2527,)


## Apply augmentation to the dataset

In [99]:
# Define the image data generator with augmentation
datagen = ImageDataGenerator(
    rotation_range=20,         # Randomly rotate images in the range 0-20 degrees
    width_shift_range=0.2,     # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,    # Randomly shift images vertically by 20% of the height
    shear_range=0.2,           # Randomly shear images
    zoom_range=0.2,            # Randomly zoom into images
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill in newly created pixels after rotation or shift
)

# Augment the dataset
augmented_images = []
augmented_labels = []

augmentation_count = int(len(imgs_data))

for img, label in zip(imgs_data, labels_data):
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    i = 0
    for batch in datagen.flow(img, batch_size=1):
        augmented_images.append(batch[0].astype('uint8'))
        augmented_labels.append(label)
        i += 1
        if i >= (augmentation_count / len(imgs_data)):
            break

# Convert lists to numpy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Combine the original and augmented data
final_imgs_data = np.concatenate((imgs_data, augmented_images), axis=0)
final_labels_data = np.concatenate((labels_data, augmented_labels), axis=0)

In [71]:
print("Size before augmentation : ",imgs_data.shape[0])
print("Size After augmentation : ",final_imgs_data.shape[0])

Size before augmentation :  2527
Size After augmentation :  5054


## Split the data

In [101]:
# Split data into training+validation (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(final_imgs_data, final_labels_data, test_size=0.2, random_state=42)

## Transfer Learning with MobileNetV2

### Normalize Data

In [103]:
# Normalize your image data
X_train = np.array(X_train) / 255.0
X_test = np.array(X_test) / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)

### Define the MobileNetV2 model

In [79]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


### Freeze the base model

In [81]:
base_model.trainable = False

### Create a new model on top

In [83]:
model = Sequential([
    base_model,
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(6, activation='softmax')  # assuming classification task
])

### Compile the model

In [105]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

### Define early stopping callback

In [87]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

### Train the model

In [113]:
# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])

MemoryError: Unable to allocate 2.27 GiB for an array with shape (4043, 224, 224, 3) and data type float32

### Evaluate the model

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

In [ ]:
# Extracting history from training
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']

# Plotting the training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

### Test

In [ ]:
# Make predictions
index_test = 110
# Assuming X_train[0] is the first image in your training data
image_to_predict = np.expand_dims(X_test[index_test], axis=0)  # Add batch dimension if necessary

# Make predictions
predictions = model.predict(image_to_predict)
predicted_class = np.argmax(predictions[0])
classes = {value:key for key,value in labels_.items()}
# Print predicted class
print(f"Predicted class: {classes[predicted_class]},\t Real class: {classes[y_test[index_test]]}")
display_image(X_test,y_test,index_test)